# A metric for Action Discrimination

The usual metric to assess predictor strength is AUC. AUC is a good measure to discriminate between who will and who will not accept an action. But in 
CDH, the customer is often given and it is more interesting to see which predictors differentiate more between the actions than others. With the help
of the BinAggregator class we can assess this. For each predictor, we first normalize the binning of the models and then determine how much the 
binnings differ from eachother.

In [ ]:
from pathlib import Path
import polars as pl
from pdstools import ADMDatamart, BinAggregator, cdh_utils
import plotly.express as px
from tqdm import tqdm

datafolder = Path("~/Downloads/cross-customers-cache/").expanduser()
dm = ADMDatamart(model_filename=str(Path(datafolder, "CDHSample_ModelSnapshots.parquet")),
                 predictor_filename=str(Path(datafolder, "CDHSample_PredictorSnapshots.parquet")),
                 subset=False)
myAggregator = BinAggregator(dm, query=(pl.col("Positives") > 200))

Get overview of the predictors including their standard metrics (AUC, and feature importance if available). We'll do a weighted average over the models.

In [ ]:
featureImportanceExpr = (
    pl.col("FeatureImportance").last()
    if "FeatureImportance" in myAggregator.all_predictorbinning.columns
    else pl.lit(0.0)
)

predictorOverview = (
    myAggregator.all_predictorbinning.group_by(
        ["PredictorName", "PredictorCategory", "ModelID"]
    )
    .agg(
        # pl.col("ModelID").unique().sort(),
        isNumeric=(pl.col("Type") == "numeric").all(),
        PredictorPerformance=pl.col("PerformanceBin").last(),
        FeatureImportance=featureImportanceExpr,
        Bins=pl.col("BinIndex").max(),
        ResponseCount=pl.col("ResponseCount").max(),
    )
    .filter(
        pl.col("Bins") > 2
    )  # I think this gets rid of the AGB models as well, which is what we want
    .sort(
        [
            "PredictorName",
            "ModelID",
        ]
    )
    .group_by(["PredictorName", "PredictorCategory"])
    .agg(
        pl.col("ModelID"),
        pl.col("isNumeric").all(),
        pl.col("Bins").mean(),
        PredictorPerformance=cdh_utils.weighted_average_polars(
            "PredictorPerformance", "ResponseCount"
        ),
        FeatureImportance=cdh_utils.weighted_average_polars(
            "FeatureImportance", "ResponseCount"
        ),
    )
    .sort(
        [
            "PredictorName",
        ]
    )
    .with_columns(
        pl.col("PredictorName").cast(pl.String),
        pl.col("PredictorCategory").cast(pl.String),
    )
)

predictorOverview.head(2).collect().to_pandas().style.hide()

Using the standard BinAggregator function to roll up all predictors over all models with a standardized binning. We could go more fancy and do roll-ups with a log or other distributions per predictor. Important is these roll ups are based on the exact same binning as the normalized binning created in the next cells.

This can be quite time consuming!

In [ ]:
preds = predictorOverview.collect()["PredictorName"].to_list()
preds

In [ ]:
overall_binning = []
for i in tqdm(range(len(preds))):
    overall_binning = overall_binning + [
        myAggregator.roll_up(preds[i], n=10, return_df=True).with_columns(
            # TODO this should be in the BinAggregator class and not be needed here
            pl.col("BinIndex").cast(pl.UInt32),
            pl.col("Models").cast(pl.UInt32),
        )
    ]
overall_binning = pl.concat(overall_binning, how="vertical_relaxed")

overall_binning.head(20).to_pandas().style

Normalize the binning for each of the predictors, for each of the models. The target binning should be the very same as used in the overall roll-up. 

The normalized binning is then joined with the overall binning for that predictor. The difference between the "lift" in a bin of the normalized binning and the lift in the same bin of the overall roll-up is the factor we're interested in. We take the difference, roll up per model ID and then average those over all models.

For the final results we join with the standard metrics (AUC and feature importance if available) for reporting and analysis.

In [ ]:
def get_modelids(pred):
    return (
        predictorOverview.filter(pl.col("PredictorName") == pred)
        .select(pl.col("ModelID").explode())
        .collect()["ModelID"]
        .to_list()
    )


def is_numeric(pred):
    return (
        predictorOverview.filter(pl.col("PredictorName") == pred)
        .select(pl.col("isNumeric"))
        .collect()
        .item()
    )


def create_normalized_binning(pred):
    if is_numeric(pred):
        target = myAggregator.create_empty_numbinning(pred, n=10)
        return pl.concat(
            [
                myAggregator.accumulate_num_binnings(
                    pred,
                    [id],
                    target.clone(),
                    # TODO the cast should be in the library
                ).with_columns(
                    pl.col("BinIndex").cast(pl.UInt32),
                    ModelID=pl.lit(id),
                )
                for id in get_modelids(pred)
            ],
            how="vertical_relaxed",
        )
    else:
        symbols = myAggregator.create_symbol_list(
            pred, n_symbols=10, musthave_symbols=[]
        )
        return pl.concat(
            [
                # TODO the cast should be in the library
                myAggregator.accumulate_sym_binnings(pred, [id], symbols).with_columns(
                    pl.col("BinIndex").cast(pl.UInt32), ModelID=pl.lit(id)
                )
                for id in get_modelids(pred)
            ],
            how="vertical_relaxed",
        )


normalized_binning = []
for i in tqdm(range(len(preds))):
    normalized_binning = normalized_binning + [create_normalized_binning(preds[i])]

predictor_metrics = (
    (
        pl.concat(
            normalized_binning,
            how="vertical_relaxed",
        )
        .join(
            overall_binning.select(["PredictorName", "BinIndex", "Lift"]),
            on=["PredictorName", "BinIndex"],
            how="left",
            suffix="_overall",
        )
        .with_columns(
            Distance=(
                pl.col("Lift_overall") - pl.col("Lift")
            ).abs()  # or, ratio, or, something else...
        )
    )
    .group_by(["PredictorName", "ModelID"])
    .agg(Distance=pl.col("Distance").mean())
    .group_by("PredictorName")
    .agg(ACDISC=pl.col("Distance").mean())
    .join(
        predictorOverview.select(
            [
                "PredictorName",
                "PredictorCategory",
                "PredictorPerformance",
                "FeatureImportance",
            ]
        ).collect(),
        on="PredictorName",
        how="left",
    )
    .sort("PredictorName")
)

predictor_metrics.to_pandas().style.hide()

In [ ]:
import plotly.express as px

px.scatter(
    predictor_metrics,
    x="PredictorPerformance",
    y="ACDISC",
    color="PredictorCategory",
    hover_name="PredictorName",
)